In [1]:
import numpy as np

In [2]:
def gibbs_step_param(data, hyper, var, sigm, nbeps, qq):
    # Étape de Gibbs pour la mise à jour des paramètres
    p = len(data)
    thetc = np.zeros(p)
    dists = 0
    
    for i in range(p):
        # Générer des échantillons pour les paramètres
        thettest = np.random.normal(hyper, var, nbeps)
        # Calculer la moyenne des échantillons pour chaque observation
        test = np.mean(np.random.normal(thettest[:, None], sigm, (nbeps, len(data))), axis=0)
        # Calculer la distance entre les observations et les simulations
        dist = np.abs(test - data[i])
        # Mettre à jour les paramètres en choisissant la simulation avec la plus petite distance
        thetc[i] = thettest[np.argmin(dist)]
        dists += np.min(dist)
    
    return thetc, dists


In [3]:

def gibbshyper(thet, nbeps2, var):
    # Étape de Gibbs pour la mise à jour de l'hyperparamètre, avec prior uniforme
    res = np.random.uniform(-4, 4, nbeps2)
    # Calculer la moyenne des échantillons pour chaque paramètre
    test = np.mean(np.random.normal(res[:, None], var, (nbeps2, len(thet))), axis=0)
    # Calculer la distance entre la moyenne des échantillons et la moyenne des paramètres
    dist = np.abs(test - np.mean(thet))
    # Mettre à jour l'hyperparamètre en choisissant la simulation avec la plus petite distance
    return res[np.argmin(dist)], np.min(dist)


In [9]:

def gibbs_abc(data, thetini, hyperini, sigm, var, nbeps1, nbeps2, nbpts, qq):
    # Algorithme complet de Gibbs ABC
    reshyper = np.zeros(nbpts + 1)
    resparam = np.zeros((len(thetini), nbpts + 1))
    reshyper[0] = hyperini
    resparam[:, 0] = thetini
    resdist = np.zeros(nbpts)
    
    for i in range(1, nbpts + 1):
        resdist[i - 1] = 0
        # Mettre à jour les paramètres
        VV, dist_param = gibbs_step_param(data, reshyper[i - 1], var, sigm, nbeps1, qq)
        resparam[:, i] = VV
        resdist[i - 1] += dist_param
        # Mettre à jour l'hyperparamètre
        WW, dist_hyper = gibbshyper(resparam[:, i], nbeps2, var)
        reshyper[i] = WW
    
    return reshyper, resparam, resdist


In [10]:

# Exemple d'utilisation :
data = np.array([3.5, 4.2, 5.1, 6.3])  # données observées
thetini = np.array([1.0, 2.0, 3.0, 4.0])  # valeurs initiales des paramètres
hyperini = 0.0  # valeur initiale de l'hyperparamètre
sigm = 0.1  # écart type des observations
var = 0.5  # variance des paramètres
nbeps1 = 1000  # nombre d'échantillons pour les paramètres
nbeps2 = 1000  # nombre d'échantillons pour l'hyperparamètre
nbpts = 10  # nombre d'itérations de l'algorithme
qq = 0.9  # quantile pour la tolérance

# Appel de l'algorithme Gibbs ABC
reshyper, resparam, resdist = gibbs_abc(data, thetini, hyperini, sigm, var, nbeps1, nbeps2, nbpts, qq)

# Affichage des résultats
print("Estimations des paramètres :", resparam[:, -1])
print("Estimation de l'hyperparamètre :", reshyper[-1])
print("Distances calculées à chaque itération :", resdist)


Estimations des paramètres : [-3.7279946  -3.96611515 -3.52297185 -4.44193806]
Estimation de l'hyperparamètre : -1.8925754426362902
Distances calculées à chaque itération : [19.10195956 27.24648106 34.15857368 34.16378373 22.26085035 25.89245247
  8.48020515 12.23388897 16.36415865 34.09458773]
